In [19]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# Carregando os dados
data = pd.read_csv("winequality.csv")

# Separando as features (X) e os rótulos (y)
X = data.drop('quality', axis=1).values
y = data['quality'].values

# Dividindo o dataset em conjunto de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))


In [5]:
def k_nearest_neighbors(X_train, X_test, y_train, k=3):
    y_pred = []
    for test_point in X_test:
        # Calculando as distâncias para todos os pontos de treinamento
        distances = [euclidean_distance(test_point, train_point) for train_point in X_train]

        # Obtendo os índices dos k vizinhos mais próximos
        k_neighbors_indices = np.argsort(distances)[:k]

        # Obtendo os rótulos dos vizinhos mais próximos
        k_neighbors_labels = [y_train[i] for i in k_neighbors_indices]

        # Fazendo a previsão para o ponto de teste: a classe mais comum entre os vizinhos mais próximos
        prediction = np.bincount(k_neighbors_labels).argmax()
        y_pred.append(prediction)

    return np.array(y_pred)


In [7]:
# Definindo o valor de k (número de vizinhos mais próximos)
k = 3

# Fazendo as previsões usando o k-NN
y_pred_knn = k_nearest_neighbors(X_train, X_test, y_train, k)

# Calculando a acurácia do k-NN
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Acurácia do k-NN:", accuracy_knn)


Acurácia do k-NN: 0.33615384615384614


In [8]:
from sklearn.ensemble import RandomForestRegressor

# Criando o modelo de Random Forest
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinando o modelo com o conjunto de treinamento
random_forest.fit(X_train, y_train)

# Obtendo a importância das features
feature_importance = random_forest.feature_importances_

# Criando um DataFrame para visualizar a importância das features
importance_df = pd.DataFrame({'Feature': data.columns[:-1], 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Importância das features (Random Forest):\n", importance_df)


Importância das features (Random Forest):
                  Feature  Importance
11               alcohol    0.251824
2       volatile acidity    0.117817
6    free sulfur dioxide    0.082968
10             sulphates    0.074930
7   total sulfur dioxide    0.072466
4         residual sugar    0.066937
9                     pH    0.061879
5              chlorides    0.058888
3            citric acid    0.055962
8                density    0.052647
1          fixed acidity    0.052399
0             Unnamed: 0    0.050493
12               quality    0.000789


In [13]:
top_3_features = importance_df['Feature'][:3].values

# Filtrando o conjunto de treinamento e teste com as três features mais importantes
X_train_top3 = X_train[:, importance_df.index[:3]]
X_test_top3 = X_test[:, importance_df.index[:3]]

In [14]:
def map_quality_label(quality):
    # Mapear a qualidade do vinho para as categorias: 'baixa', 'média' e 'alta'
    if quality <= 5:
        return 'baixa'
    elif quality <= 7:
        return 'média'
    else:
        return 'alta'

# Fazendo as previsões usando o k-NN nas três features mais importantes
knn_top3 = k_nearest_neighbors(X_train_top3, X_test_top3, y_train, k)

# Mapeando as previsões de qualidade para as categorias
predicted_quality_labels = np.array([map_quality_label(quality) for quality in knn_top3])

# Mapeando as categorias reais do conjunto de teste
true_quality_labels = np.array([map_quality_label(quality) for quality in y_test])

# Calculando a acurácia do k-NN com as três features mais importantes
accuracy_knn_top3 = accuracy_score(true_quality_labels, predicted_quality_labels)
print("Acurácia do k-NN (com as 3 features mais importantes):", accuracy_knn_top3)


Acurácia do k-NN (com as 3 features mais importantes): 0.666923076923077


In [15]:
# Mostrar as primeiras observações do conjunto de teste e a nova classificação
result_df = pd.DataFrame({
    'Features': X_test_top3.tolist(),
    'True_Quality': true_quality_labels,
    'Predicted_Quality': predicted_quality_labels
})

print(result_df.head())


              Features True_Quality Predicted_Quality
0    [9.9, 0.47, 36.0]        baixa             média
1  [11.6, 0.405, 10.0]        baixa             baixa
2   [11.3, 0.56, 24.0]        baixa             média
3   [11.8, 0.44, 60.0]        média             média
4     [9.6, 0.51, 8.0]        média             baixa


In [20]:
X = data.drop('wine_is_red', axis=1).values
y = data['wine_is_red'].values

# Dividindo o dataset em conjunto de treinamento e teste (80% treinamento, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o modelo de Random Forest Classifier
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinando o modelo com o conjunto de treinamento
random_forest_classifier.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = random_forest_classifier.predict(X_test)

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

# Imprimindo o relatório de classificação (com métricas de precisão, recall, f1-score, etc.)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

Acurácia do modelo: 0.9930769230769231
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       979
           1       0.99      0.98      0.99       321

    accuracy                           0.99      1300
   macro avg       0.99      0.99      0.99      1300
weighted avg       0.99      0.99      0.99      1300

